## IGSE 方程

$$
P_{\text{core}} = \frac{1}{T} \int_0^T k \left| \frac{dB}{dt} \right|^\alpha \left( \Delta B \right)^{\beta - \alpha} , dt
$$

In [21]:
import time

import pandas as pd
import numpy as np

d1 = pd.read_csv("data/材料1.csv")
d2 = pd.read_csv("data/材料2.csv")
d3 = pd.read_csv("data/材料3.csv")
d4 = pd.read_csv("data/材料4.csv")

data = pd.concat([d1, d2, d3, d4])

pd.set_option('future.no_silent_downcasting', True)

data['励磁波形'] = data['励磁波形'].replace({'正弦波': 1, '三角波': 2, '梯形波': 3})

magnetic_flux_density = data.iloc[:, 4:-1].values
# print(magnetic_flux_density.shape)

lst = ['温度', '频率', "磁芯损耗", '励磁波形']
for i in range(1024):
    lst.append(i)
# lst.append('材料')
data.columns = lst


filter_data = data[['温度', '频率', '励磁波形', '磁芯损耗']]


In [22]:
import numpy as np
from scipy.optimize import curve_fit
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# 提取相关列：频率(频率)、磁芯损耗(磁芯损耗)、以及dB/dt数据(从第5列到第1023列)
frequencies = data['频率']
core_loss = data['磁芯损耗']
db_dt = data.iloc[:, 5:]  # 假设第5列及以后是 dB/dt 的数据

# 根据图片中的公式定义核心损耗函数
def core_loss_function_with_ki(db_dt, k, alpha, beta):
    # 计算 ΔB
    delta_b = np.max(db_dt, axis=1) - np.min(db_dt, axis=1)
    
    # 定义 ki 基于公式的数值积分
    def ki_integral(alpha, beta):
        theta = np.linspace(0, 2 * np.pi, 1000)  # 用数值方法逼近积分
        integrand = (np.abs(np.cos(theta)) ** alpha) * (2 ** (beta - alpha))
        return np.trapz(integrand, theta)

    # 计算 ki
    ki = k / ((2 * np.pi) ** (alpha - 1)) * ki_integral(alpha, beta)

    # 计算核心损耗方程中的积分项
    integrand = np.abs(db_dt) ** alpha
    integral = np.trapz(integrand, axis=1)  # 在时间轴上进行数值积分

    # 返回预测的磁芯损耗
    return (ki * (delta_b ** (beta - alpha)) * integral) / db_dt.shape[1]

# 初始猜测值 k, alpha, beta
initial_guess = [1, 1, 1]

# 将数据集划分为 80% 训练集和 20% 测试集
train_db_dt, test_db_dt, train_core_loss, test_core_loss = train_test_split(db_dt, core_loss, test_size=0.2, random_state=42)

# 使用训练集进行拟合，找到 k, alpha, beta
popt_new, _ = curve_fit(lambda db_dt_values, k, alpha, beta: core_loss_function_with_ki(db_dt_values, k, alpha, beta), 
                        train_db_dt.values, train_core_loss, p0=initial_guess)

# 提取优化后的参数
k_new, alpha_new, beta_new = popt_new

# 输出拟合结果
print(f"优化后的参数: k = {k_new}, alpha = {alpha_new}, beta = {beta_new}")

# 使用拟合得到的参数预测测试集的磁芯损耗
predicted_test_core_loss = core_loss_function_with_ki(test_db_dt.values, k_new, alpha_new, beta_new)

# 计算测试集上的评估指标
mse = mean_squared_error(test_core_loss, predicted_test_core_loss)
mae = mean_absolute_error(test_core_loss, predicted_test_core_loss)
r2 = r2_score(test_core_loss, predicted_test_core_loss)
mape = np.mean(np.abs((test_core_loss - predicted_test_core_loss) / test_core_loss)) * 100

# 输出评估结果
print(f"MSE: {mse}")
print(f"MAE: {mae}")
print(f"R² Score: {r2}")
print(f"MAPE: {mape}")

/var/folders/gy/6wf0tc3n7276h9vd9dqcpkvw0000gn/T/ipykernel_10410/3853444980.py:26: RuntimeWarning: divide by zero encountered in power
  integrand = np.abs(db_dt) ** alpha
/var/folders/gy/6wf0tc3n7276h9vd9dqcpkvw0000gn/T/ipykernel_10410/3853444980.py:26: RuntimeWarning: overflow encountered in power
  integrand = np.abs(db_dt) ** alpha
/var/folders/gy/6wf0tc3n7276h9vd9dqcpkvw0000gn/T/ipykernel_10410/3853444980.py:30: RuntimeWarning: overflow encountered in multiply
  return (ki * (delta_b ** (beta - alpha)) * integral) / db_dt.shape[1]


优化后的参数: k = 12.302067501266475, alpha = 1.6875308706765046e-06, beta = -0.8500960172702441
MSE: 171053696195.74435
MAE: 190232.37906500648
R² Score: -0.2714102887852454
MAPE: 93.91905451148698
